In [31]:
# Importing libraries
import nltk
import numpy as np
import pandas as pd
import random
from sklearn.model_selection import train_test_split
import pprint, time
 
nltk.download('treebank')
nltk.download('universal_tagset')
nltk_data = list(nltk.corpus.treebank.tagged_sents(tagset='universal'))
print(nltk_data[:2])

[nltk_data] Downloading package treebank to
[nltk_data]     /home/abenezer121/nltk_data...
[nltk_data]   Package treebank is already up-to-date!
[nltk_data] Downloading package universal_tagset to
[nltk_data]     /home/abenezer121/nltk_data...
[nltk_data]   Package universal_tagset is already up-to-date!


[[('Pierre', 'NOUN'), ('Vinken', 'NOUN'), (',', '.'), ('61', 'NUM'), ('years', 'NOUN'), ('old', 'ADJ'), (',', '.'), ('will', 'VERB'), ('join', 'VERB'), ('the', 'DET'), ('board', 'NOUN'), ('as', 'ADP'), ('a', 'DET'), ('nonexecutive', 'ADJ'), ('director', 'NOUN'), ('Nov.', 'NOUN'), ('29', 'NUM'), ('.', '.')], [('Mr.', 'NOUN'), ('Vinken', 'NOUN'), ('is', 'VERB'), ('chairman', 'NOUN'), ('of', 'ADP'), ('Elsevier', 'NOUN'), ('N.V.', 'NOUN'), (',', '.'), ('the', 'DET'), ('Dutch', 'NOUN'), ('publishing', 'VERB'), ('group', 'NOUN'), ('.', '.')]]


In [33]:
#split the training and testing set into 80 20 ratio
train_set,test_set =train_test_split(nltk_data,train_size=0.80,test_size=0.20,random_state = 101)

In [35]:
# create list of train and test tagged words
train_tagged_words = [ tup for sent in train_set for tup in sent ]
test_tagged_words = [ tup for sent in test_set for tup in sent ]


print(test_tagged_words[:3])

[('The', 'DET'), ('company', 'NOUN'), ('said', 'VERB')]


In [21]:
#use set datatype to check how many unique tags are present in training data
tags = {tag for word,tag in train_tagged_words}
print(len(tags))
print(tags)
 
# check total words in vocabulary
vocab = {word for word,tag in train_tagged_words}

12
{'DET', 'VERB', 'ADV', 'CONJ', 'ADP', '.', 'ADJ', 'PRON', 'PRT', 'NUM', 'NOUN', 'X'}


In [22]:
# compute Emission Probability
def word_given_tag(word, tag, train_bag = train_tagged_words):
    tag_list = [pair for pair in train_bag if pair[1]==tag]
    count_tag = len(tag_list)#total number of times the passed tag occurred in train_bag
    w_given_tag_list = [pair[0] for pair in tag_list if pair[0]==word]
#now calculate the total number of times the passed word occurred as the passed tag.
    count_w_given_tag = len(w_given_tag_list)
 
     
    return (count_w_given_tag, count_tag)

In [24]:
# compute  Transition Probability
def t2_given_t1(t2, t1, train_bag = train_tagged_words):
    tags = [pair[1] for pair in train_bag]
    count_t1 = len([t for t in tags if t==t1])
    count_t2_t1 = 0
    for index in range(len(tags)-1):
        if tags[index]==t1 and tags[index+1] == t2:
            count_t2_t1 += 1
    return (count_t2_t1, count_t1)

In [25]:
tags_matrix = np.zeros((len(tags), len(tags)), dtype='float32')
for i, t1 in enumerate(list(tags)):
    for j, t2 in enumerate(list(tags)): 
        tags_matrix[i, j] = t2_given_t1(t2, t1)[0]/t2_given_t1(t2, t1)[1]
 
tags_df = pd.DataFrame(tags_matrix, columns = list(tags), index=list(tags))
display(tags_df)

,DET,VERB,ADV,CONJ,ADP,.,ADJ,PRON,PRT,NUM,NOUN,X
DET,0.006037,0.040247,0.012074,0.000431,0.009918,0.017393,0.206411,0.003306,0.000287,0.022855,0.635906,0.045134
VERB,0.133610,0.167956,0.083886,0.005433,0.092357,0.034807,0.066390,0.035543,0.030663,0.022836,0.110589,0.215930
ADV,0.071373,0.339022,0.081458,0.006982,0.119472,0.139255,0.130721,0.012025,0.014740,0.029868,0.032196,0.022886
CONJ,0.123491,0.150384,0.057080,0.000549,0.055982,0.035126,0.113611,0.060373,0.004391,0.040615,0.349067,0.009330
ADP,0.320931,0.008479,0.014553,0.001012,0.016958,0.038724,0.107062,0.069603,0.001266,0.063275,0.323589,0.034548
.,0.172192,0.089690,0.052569,0.060079,0.092908,0.092372,0.046132,0.068769,0.002789,0.078210,0.218539,0.025641
ADJ,0.005243,0.011456,0.005243,0.016893,0.080583,0.066019,0.063301,0.000194,0.011456,0.021748,0.696893,0.020971
PRON,0.009567,0.484738,0.036902,0.005011,0.022323,0.041913,0.070615,0.006834,0.014123,0.006834,0.212756,0.088383
PRT,0.101370,0.401174,0.009393,0.002348,0.019569,0.045010,0.082975,0.017613,0.001174,0.056751,0.250489,0.012133
NUM,0.003570,0.020707,0.003570,0.014281,0.037487,0.119243,0.035345,0.001428,0.026062,0.184220,0.351660,0.202428


In [26]:
def Viterbi(words, train_bag = train_tagged_words):
    state = []
    T = list(set([pair[1] for pair in train_bag]))
     
    for key, word in enumerate(words):
        #initialise list of probability column for a given observation
        p = [] 
        for tag in T:
            if key == 0:
                transition_p = tags_df.loc['.', tag]
            else:
                transition_p = tags_df.loc[state[-1], tag]
                 
            # compute emission and state probabilities
            emission_p = word_given_tag(words[key], tag)[0]/word_given_tag(words[key], tag)[1]
            state_probability = emission_p * transition_p    
            p.append(state_probability)
             
        pmax = max(p)
        # getting state for which probability is maximum
        state_max = T[p.index(pmax)] 
        state.append(state_max)
    return list(zip(words, state))

In [27]:
# Let's test our Viterbi algorithm on a few sample sentences of test dataset
random.seed(1234)      #define a random seed to get same sentences when run multiple times
 
# choose random 10 numbers
rndom = [random.randint(1,len(test_set)) for x in range(10)]
 
# list of 10 sents on which we test the model
test_run = [test_set[i] for i in rndom]
 
# list of tagged words
test_run_base = [tup for sent in test_run for tup in sent]
 
# list of untagged words
test_tagged_words = [tup[0] for sent in test_run for tup in sent]

In [28]:
#Here We will only test 10 sentences to check the accuracy
#as testing the whole training set takes huge amount of time
start = time.time()
tagged_seq = Viterbi(test_tagged_words)
end = time.time()
difference = end-start
 
print("Time taken in seconds: ", difference)
 
# accuracy
check = [i for i, j in zip(tagged_seq, test_run_base) if i == j] 
 
accuracy = len(check)/len(tagged_seq)
print('Viterbi Algorithm Accuracy: ',accuracy*100)

Time taken in seconds:  37.67053031921387
Viterbi Algorithm Accuracy:  93.77990430622009
